In [ ]:
orientation_axis_x = 0
orientation_axis_y = 1
orientation_axis_z = 2

In [ ]:
from pyrep.vrep.vrep import simxGetObjectOrientation, simxGetObjectHandle, simxGetFloatSignal
from pyrep.vrep import vrep as v

import math

In [ ]:
rclass='PioneerP3DX_base'
print('Loading class: {}'.format(rclass))
class PioneerP3DX_base:

    def __init__(self, api: VRep):
        self._api = api
        self._left_motor = api.joint.with_velocity_control("Pioneer_p3dx_leftMotor")
        self._right_motor = api.joint.with_velocity_control("Pioneer_p3dx_rightMotor")
        self._sensor_ultrasonic_left = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor3")
        self._sensor_ultrasonic_right = api.sensor.proximity("Pioneer_p3dx_ultrasonicSensor6")
       
        res, self._handle = simxGetObjectHandle(self.id, 'Pioneer_p3dx', vrep.simx_opmode_oneshot_wait)

    def get_orientation(self):
        #http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctions.htm#simxGetObjectOrientation
        #Returns a value between +/-pi
        return simxGetObjectOrientation(self.id, self._handle, -1, v.simx_opmode_streaming)[1]

        
print('This is a base class for the {} model\n'.format(eval(rclass).__name__ ))

In [ ]:
rclass='PioneerP3DX'
print('Loading class: {}'.format(rclass))
class PioneerP3DX(PioneerP3DX_base):

    def __init__(self, api: VRep):
        self._api = api
        self.id = api._id
        
        #Inherit init settings from parent class
        super(PioneerP3DX, self).__init__(api)
   

    def getvalleft(self):
        return simxGetFloatSignal(self.id,'leftEncoder',v.simx_opmode_streaming)#simx_opmode_streaming, simx_opmode_buffer)

    def getvalright(self):
        return simxGetFloatSignal(self.id,'rightEncoder',v.simx_opmode_streaming)#simx_opmode_streaming, simx_opmode_buffer)

    def get_orientation_degrees(self,axis=orientation_axis_z):
        radians = self.get_orientation()
        #Provide an option to return orientation in degrees about each axis
        return [math.degrees(r) for r in radians] if axis==-1 else math.degrees(radians[axis])

    def set_two_motor(self, left: float, right: float):
        self._left_motor.set_target_velocity(left)
        self._right_motor.set_target_velocity(right)

    def rotate_right(self, speed=2.0):
        self.set_two_motor(speed, -speed)

    def rotate_left(self, speed=2.0):
        self.set_two_motor(-speed, speed)

    def move_forward(self, speed=2.0):
        self.set_two_motor(speed, speed)

    def move_backward(self, speed=2.0):
        self.set_two_motor(-speed, -speed)
    
    def ultrasonic_right_length(self):
        return self._sensor_ultrasonic_right.read()[1].distance()

    def ultrasonic_left_length(self):
        return self._sensor_ultrasonic_left.read()[1].distance()

methods = [method for method in dir(eval(rclass)) if not method.startswith('_')]
print('Methods available in {}:\n\t{}\n'.format(eval(rclass).__name__ , '\n\t'.join(methods)))

In [ ]:
rclass='PioneerP3DXL'
print('Loading class: {}'.format(rclass))
#https://github.com/Troxid/vrep-api-python/blob/master/examples/line_follower_pioneer.py
class PioneerP3DXL(PioneerP3DX_base):

    def __init__(self, api: VRep):
        self._api = api
        self.id = api._id
        
        super(PioneerP3DXL, self).__init__(api)
        
        self._sensor_rgb_left = api.sensor.vision("LeftRGBSensor")  # type: VisionSensor
        self._sensor_rgb_right = api.sensor.vision("RightRGBSensor")  # type: VisionSensor

    
    def set_two_motor(self, left: float, right: float):
        self._left_motor.set_target_velocity(left)
        self._right_motor.set_target_velocity(right)

    def rotate_right(self, speed=2.0):
        self.set_two_motor(speed, -speed)

    def rotate_left(self, speed=2.0):
        self.set_two_motor(-speed, speed)

    def move_forward(self, speed=2.0):
        self.set_two_motor(speed, speed)

    def move_backward(self, speed=2.0):
        self.set_two_motor(-speed, -speed)

    def color_right(self) -> int:
        image = self._sensor_rgb_right.raw_image(is_grey_scale=True)  # type: List[int]
        average = sum(image) / len(image)
        return average

    def color_left(self) -> int:
        image = self._sensor_rgb_left.raw_image(is_grey_scale=True)  # type: List[int]
        average = sum(image) / len(image)
        return average
    
    def ultrasonic_right_length(self):
        return self._sensor_ultrasonic_right.read()[1].distance()

    def ultrasonic_left_length(self):
        return self._sensor_ultrasonic_left.read()[1].distance()


methods = [method for method in dir(eval(rclass)) if not method.startswith('_')]
print('Methods available in {}:\n\t{}\n'.format(eval(rclass).__name__ , '\n\t'.join(methods)))

## Widget Displays

In [2]:
from IPython.display import display
import ipywidgets as widgets

sensorText1 = widgets.Text(value='', 
                          disabled=True, 
                          description='')

sensorText2 = widgets.Text(value='', 
                             disabled=True, 
                             description='')

print("The following text widgets are available for display: sensorText1, sensorText2")

The following text widgets are available for display: sensorText1, sensorText2
